# Prepare Ribbon for SPM Simulation with AiiDA

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as ipw
from fileupload import FileUploadWidget
import nglview

from tempfile import NamedTemporaryFile
import numpy as np
from numpy.linalg import norm
import scipy.stats
import ase
import ase.io
from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm import DataFactory

## Step 1: Upload a .mol file from ChemSketch or select example

In [ ]:
def on_file_upload(c):
    global atoms
    tmp = NamedTemporaryFile(suffix=file_upload.filename)
    f = open(tmp.name, "w")
    f.write(file_upload.data)
    f.close()
    atoms = ase.io.read(tmp.name)
    tmp.close()
    setup_new_atoms()
    

def on_click_example(b):
    global atoms
    if b == btn_eg1:
        atoms = ase.io.read("chevron.mol")
    if b == btn_eg2:
        atoms = ase.io.read("polymer.mol")
    if b == btn_eg3:
        atoms = ase.io.read("5_1GNR.mol")
    setup_new_atoms()
    
    
#TODO: FileUploadWidget doesn't fire event when same file is uploaded twice
file_upload = FileUploadWidget("Upload Structure")
file_upload.observe(on_file_upload, names='data')

btn_eg1 = ipw.Button(description='chevron.mol')
btn_eg2 = ipw.Button(description='polymer.mol')
btn_eg3 = ipw.Button(description='p5_1GNR.mol')
btn_eg1.on_click(on_click_example)
btn_eg2.on_click(on_click_example)
btn_eg3.on_click(on_click_example)
newatoms_out = ipw.Output()

display(ipw.HBox([file_upload, btn_eg1, btn_eg2, btn_eg3]), newatoms_out)

In [ ]:
def setup_new_atoms():
    global atoms, selection, viewer
    
    with newatoms_out:
        clear_output()
        print("Found %i atoms."%len(atoms))
        print("Fixing scale....")
        fix_scale(atoms)

        # remove old components
        if hasattr(viewer, "component_0"):
            viewer.component_0.remove_ball_and_stick()
            #viewer.component_0.remove_unitcell()
            cid = viewer.component_0.id
            viewer.remove_component(cid)

        # empty selection
        selection = set()

        # add new component
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        #viewer.add_unitcell()
        viewer.center_view()

In [ ]:
def fix_scale(atoms):
    # set bounding box as cell
    cx = 1.5 * (np.amax(atoms.positions[:,0]) - np.amin(atoms.positions[:,0]))
    cy = 1.5 * (np.amax(atoms.positions[:,1]) - np.amin(atoms.positions[:,1]))
    cz = 15.0
    atoms.cell = (cx, cy, cz)
    atoms.pbc = (True,True,True)
    
    # calculate all atom-atom distances
    n = len(atoms)
    dists = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            dists[i,j] = norm(atoms[i].position - atoms[j].position)
            
    # find bond distances to closest neighbor
    dists += np.diag([np.inf]*n) # don't consider diagonal
    bonds = np.amin(dists, axis=1)
    
    # average bond distance
    avg_bond = float(scipy.stats.mode(bonds)[0])
    
    # scale box to match equilibrium carbon-carbon bond distance
    cc_eq = 1.4313333333
    s = cc_eq / avg_bond
    print("Scaled positions by: %f"%s)
    atoms.set_cell([s*cx, s*cy, cz], scale_atoms=True)
    atoms.center()

## Step 2: Select two equivalent atoms to guide the unit cell construction

In [ ]:
def on_picked(c):
    global viewer, selection
        
    if 'atom' not in viewer.picked.keys():
        return # did not click on atom
    with picked_out:
        clear_output()
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.add_ball_and_stick()
        #viewer.add_unitcell()

        idx = viewer.picked['atom']['index']

        # toggle
        if idx in selection:
            selection.remove(idx)
        else:
            selection.add(idx)

        #if(selection):
        sel_str = ",".join([str(i) for i in sorted(selection)])
        print("Selected atoms: "+ sel_str)
        viewer.add_representation('ball+stick', selection="@"+sel_str, color='red', aspectRatio=3.0)
        #else:
        #    print ("nothing selected")
        viewer.picked = {} # reset, otherwise immidiately selecting same atom again won't create change event

selection = set()    
viewer = nglview.NGLWidget()
viewer.observe(on_picked, names='picked')
picked_out = ipw.Output()
display(viewer, picked_out)

## Step 3: Construct unit cell and check results

In [ ]:
def on_click_cell(b):
    global viewer2, atoms2
    
    with clickcell_out:
        clear_output()
        atoms2 = None
        if hasattr(viewer2, "component_0"):
            viewer2.component_0.remove_ball_and_stick()
            viewer2.component_0.remove_unitcell()
            cid = viewer2.component_0.id
            viewer2.remove_component(cid)   

        if len(selection) != 2:
            print("You must select exactly two atoms")
        else:
            atoms2 = atoms.copy()
            construct_cell(atoms2, selection)

            # display in second viewer
            viewer2.add_component(nglview.ASEStructure(atoms2)) # adds ball+stick
            #viewer2.add_ball_and_stick()
            viewer2.add_unitcell()
            viewer2.center_view()

btn_cell = ipw.Button(description='Construct Cell')
btn_cell.on_click(on_click_cell)
clickcell_out = ipw.Output()
display(btn_cell, clickcell_out)

In [ ]:
atoms2 = None
viewer2 = nglview.NGLWidget()
display(viewer2)

In [ ]:
def construct_cell(atoms, selection):
    id1, id2 = selection
    
    p1 = [atoms[id1].x, atoms[id1].y]
    p0 = [atoms[id2].x, atoms[id2].y]
    p2 = [atoms[id2].x, atoms[id1].y]
    
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))

    #angle=np.degrees(angle)

    yrange = np.amax(atoms.positions[:,1])-np.amin(atoms.positions[:,1])
    
    cx = norm(v0)
    cy = 15.0 + yrange
    cz = 15.0
    #print np.degrees(angle),v0,v1,p0,p1
    if np.abs(angle) > 0.01:
    #   s.euler_rotate(phi=angle,theta=0,psi=0,center(x[id1],y[id1],z[id1]))
        atoms.rotate_euler(center=atoms[id1].position, phi=-angle,theta=0.0,psi=0.0)
    atoms.cell = (cx,cy,cz)
    atoms.pbc = (True,True,True)
    atoms.wrap(eps=0.001)
    atoms.center()

    #### REMOVE REDUNDANT ATOMS
    tobedel = []

    cov_radii = [covalent_radii[a.number] for a in atoms]
    nl = NeighborList(cov_radii, bothways = False, self_interaction = False)
    nl.update(atoms)

    for a in atoms:
        indices, offsets = nl.get_neighbors(a.index)
        for i, offset in zip(indices, offsets):
            dist = norm(a.position -(atoms.positions[i] + np.dot(offset, atoms.get_cell())))
            if dist < 0.4 :
                tobedel.append(atoms[i].index)

    del atoms[tobedel]
    #s.write("unit.xyz")
    #### END REMOVE REDUNDANT ATOMS

    #s.cell=[cx,cy,cz]
    #s.set_cell(s.cell/(1.54/1.41333),scale_atoms=True)
    #s.cell=[cx/scale_armchair,cy,cz]
    #s.center()



    #### ENDFIND UNIT CELL AND APPLIES IT


    #### ADD Hydrogens
    cov_radii = [covalent_radii[a.number] for a in atoms]
    nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
    nl.update(atoms)

    need_a_H = []
    for a in atoms:
        nlist=nl.get_neighbors(a.index)[0]
        if len(nlist)<3:
            if a.symbol=='C':
                need_a_H.append(a.index)

    print "Added missing Hydrogen atoms: ", need_a_H

    dCH=1.1
    for a in need_a_H:
        vec = np.zeros(3)
        indices, offsets = nl.get_neighbors(atoms[a].index)
        for i, offset in zip(indices, offsets):
            vec += -atoms[a].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
        vec = -vec/norm(vec)*dCH
        vec += atoms[a].position
        htoadd = ase.Atom('H',vec)
        atoms.append(htoadd)

## Step 4: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    global atoms2
    
    if not atoms2:
        print("Unit cell has to be constructed first.")
        return
    
    StructureData = DataFactory('structure')
    s = StructureData(ase=atoms2)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))